In [42]:
import torch
from collections import deque
import random
import copy
from torch.autograd import Variable
import numpy as np

 

In [43]:
reward_pc = torch.tensor([1, 5 , 5,3,-1, 5, 5,3,1,-2 ])
reward_npc = torch.tensor([0,-2,-3,-1,0, -2, -3, -1,1])

#index = id-1
# action_transform
action_transform = torch.tensor([[1,3,6,7],[2,4,6,8],[1,5,6,7]])

In [44]:
dialogue_mapping_NPC = {
    1:"hello witcher",
    2:"hey kid!",
    3:"How is it going on brave witcher lord?",
    4:"Witcher can you save us from the creature thats attacking us?",
    5:"Oe witcher,  defeat a monster for me.",
    6:"Lord witcher can you save us from the creature that is attacking us?",
    7:"We don't know what monster attacked us but it was vicious. It must have to huge as it slaughtered no less than a dozen wolves. Ripped their guts out, but left lost uneaten. Howls at nighhts. People are afraid to venture into the woods at night. Please lord witcher save us from the beast.",
    8:" We don't know what monster attacked us but it was vicious. It must have to huge as it slaughtered no less than a dozen wolves. Ripped their guts out, but left lost uneaten. Howls at nighhts. People are afraid to venture into the woods at night.",
    9:"No idea what monster attacked us if i knew i would be dead. It must have to huge as it slaughtered no less than a dozen wolves. Ripped their guts out, but left lost uneaten. You sure kid like you can handle it. Howls at nighhts. People are afraid to venture into the woods at night. Sure you can take care of it we dont want any more dead bodies."
}


dialogue_mapping_PC ={
    1:"Greetings",
    2:"Fine just busy fighting creatures from Void dimension",
    3:"Ok let me help you with it. What creature is it?",
    4:" I m gonna help you but i need to paid well for a contract like that. What kind of creature is it",
    5:"I don't have to bother with this work. farewell",
    6:" Ok, sir let me help you. What kind of creature is it.",
    7:"Sure it will take 500$ ",
    8:" Sure it will take 800$",
    9:"Sure it will take 1200$",
    10:"Sorry I am busy. I won't be able to do that."


}

In [45]:
def user_in(lister):
   
    
    for i in lister:

        print(dialogue_mapping_PC[i], i)
    x=input()
    return x


In [46]:
class Environment_user():
    def __init__(self):
        self.state=(0,0) #(NPC, PC)
        self.superState = 0
    def PC_action(self,action_transformed):
        if self.superState ==0:
            return 1 
            ## code to ask input from the user
            # transform into the id

        elif self.superState == 1:
            if action_transformed == 3:
                return 2
       
            else:
                lister = [3,4]
                x= user_in(lister)
                return int(x)

        elif self.superState ==2:
            return 6

        elif self.superState ==3 :
            lister = [7,8,9,10]
            x= user_in(lister)
            return int(x)  

        else :
            return -1
        
    def return_reward(self):
        return reward_npc[self.state[0]-1] + reward_pc[self.state[1]-1]


    def next_state(self,action_transformed ):
        pc_action = self.PC_action(action_transformed)
        if pc_action ==-1:
            self.state =(-1,-1)
            return self.state
        self.state = (action_transformed,pc_action)
        if self.superState == 0:
            
            self.superState =1
        elif self.superState == 1:

            if action_transformed == 2:

                self.superState = 2
            elif pc_action == 6:
                self.superState =4

            else :
                self.superState=3


        elif self.superState ==2:
            self.superState = 3

        elif self.superState ==3:
            self.superState = 4

        else:
            self.superState = 4
        
        return self.state
        
                           

        

            



        




        
        
# This is just a dummy environment 

In [47]:
class Environment():
    def __init__(self):
        self.state=(0,0)
        self.superState = 0
    def PC_action(self,action_transformed):
        if self.superState ==0:
            return 1 
            ## code to ask input from the user
            # transform into the id

        elif self.superState == 1:
            if action_transformed == 3:
                return 2
            elif action_transformed ==5:
                return -1
            else:
                lister = [3,4]
                return random.choice(lister)

        elif self.superState ==2:
            return 6

        elif self.superState ==3 :
            lister = [7,8,9,10]
            return random.choice(lister)   

        else :
            return -1
        
    def return_reward(self):
        return reward_npc[self.state[0]-1] + reward_pc[self.state[1]-1]


    def next_state(self,action_transformed ):
        pc_action = self.PC_action(action_transformed)
        if pc_action ==-1:
            self.state =(-1,-1)
            return self.state
        self.state = (action_transformed,pc_action)
        if self.superState == 0:
            
            self.superState =1
        elif self.superState == 1:

            if action_transformed == 2:

                self.superState = 2
            elif pc_action == 6:
                self.superState =4
                self.state = (-1,-1)

            else :
                self.superState=3


        elif self.superState ==2:
            self.superState = 3

        elif self.superState ==3:
            self.superState = 4

        else:
            self.superState = 4
            self.state = (-1,-1)
        #print(self.state[1])
        return self.state
        
                           

        

            



        




        
        
# This is just a dummy environment 

In [48]:
#Instantiata a environment

env = Environment_user()


In [49]:
class DQN():
    def __init__(self, n_state, n_action, n_hidden=50, lr=0.05):
        self.criterion = torch.nn.MSELoss()

        self.model = torch.nn.Sequential(
            torch.nn.Linear(n_state,n_hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(n_hidden, n_action)
        )

        self.optimizer = torch.optim.Adam(self.model.parameters(),lr)

        self.model_target = copy.deepcopy(self.model)

    def target_predict(self, s):
        with torch.no_grad():
            return self.model_target(torch.Tensor(s))

    def copy_target(self):
        self.model_target.load_state_dict(self.model.state_dict())
    
    def update(self, s,y):
        y_pred = self.model(torch.Tensor(s))
        loss = self.criterion(y_pred, Variable(torch.Tensor(y)))
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

    def predict(self, s):
        with torch.no_grad():
            return self.model(torch.Tensor(s))


            
    def replay(self, memory, replay_size, gamma):
        if len(memory) >= replay_size:
            replay_data = random.sample(memory , replay_size)
            states =[]
            td_targets =[]
            for state, action, next_state, reward, is_done in replay_data:
                states.append(state)
                q_values = self.predict(state).tolist()
                if is_done:
                    q_values[action] = reward
                else:
                    q_values_next = self.target_predict(next_state).detach()
                    q_values[action] = reward + gamma* torch.max(q_values_next).item()

                td_targets.append(q_values)

            states = np.array(states)
            self.update(states,td_targets)

       



In [50]:
def gen_epsilon_greed_policy(estimator,epsilon, n_action ):
    def policy_function(state):
        if random.random() < epsilon:
            return random.randint(0,n_action-1)

        else:
            q_values = estimator.predict(state)
            return torch.argmax(q_values).item()

    return policy_function


In [51]:
n_state = 2
n_action = 3
n_hidden =50
lr = 0.01
dqn = DQN(n_state, n_action, n_hidden, lr)

In [52]:
memory = deque(maxlen=10000)

In [71]:
def q_learning(env, estimator , n_episode, replay_size, target_update=10, gamma=1.0, epsilon=0.1 , epsilon_decay=0.99):
    for episode in range(n_episode):
        if episode % target_update ==0 :
            estimator.copy_target()
        policy = gen_epsilon_greed_policy(estimator, epsilon, n_action)
        env.state=(0,0)
        env.superState = 0
        state = env.state

        is_done = False
        print(state)

        while not is_done:
            
            action = policy(state)
            
            transform_action = action_transform[action,env.superState]
             
           # print("npc:", transform_action)
            next_state = env.next_state(transform_action)
            print(dialogue_mapping_NPC[next_state[0].item()])
            print(dialogue_mapping_PC[next_state[1]])
            reward =env.return_reward()
            total_reward_episode[episode] += reward

            

            memory.append((state, action, next_state, reward, is_done))
            if env.superState == 4:
                is_done = True
            if is_done:
                print("end episode")
                break

            estimator.replay(memory, replay_size, gamma)
            state = next_state
            print(state)

        print('episode: {}, total reward : {}, epsilon:{}'.format(episode, total_reward_episode[episode], epsilon))
        epsilon = max(epsilon * epsilon_decay, 0.01)











In [72]:
n_episode = 1000
replay_size = 20 

target_update = 10 


In [73]:
total_reward_episode = [0] * n_episode
q_learning(env, dqn, n_episode, replay_size, target_update, gamma=0.9, epsilon = 1)

(0, 0)
hello witcher
Greetings
(tensor(1), 1)
Ok let me help you with it. What creature is it? 3
 I m gonna help you but i need to paid well for a contract like that. What kind of creature is it 4


ValueError: invalid literal for int() with base 10: ''